## Applications - Managing a company's inventory

This lecture provides a framework for managing inventory in a manufacturing based company (i.e. one that produces material goods such as soap, food, or chemicals).

---

# Table of Contents


#### <a href='#1'>A Primer on Inventory</a>
#### <a href='#2'>A primer on product demand</a>
#### <a href='#3'>Data Manipulation </a>
#### <a href='#4'>A note on Weeks of Supply</a>
#### <a href='#5'>A note on production planning</a>
#### <a href='#6'>Preliminary Results & Commentary</a>


<a id='1'></a>
## A primer on inventory.

Inventory is a term accountants use to describe the assets on hand that a company intends to sell as a part of its primary business activity.  For example, when a box factory sells boxes to a meat packer, it has sold its inventory.  Conversely, when that same box factory sells off some of it's receivables (I.O.U.s from other companies) it is NOT selling its inventory.  

For most manufacturing based businesses, inventory typically ties up more capital throughout a calendar year than any other asset with the exception of PP&E (property, plant & equipment), so most of these companies have a huge interest in managing it efficiently.  In this lecture, we will examine a hypothical company's current inventory position, discuss why it may be good or bad, propose a possible path for improvement, and conclude with a discussion on why inventory optimization is a critical component to the success of modern companies.  



In [1]:
#As always, import all needed libraries first
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
#Read in inventory data
inventory_data = pd.read_csv('Inventory_Data.csv')

In [3]:
#Get a feel for the structure of the data
inventory_data.head()

,Warehouse,SKU_ID,Product_Family,Inventory as of 1/1/22,Cost
0,W_B,1395072,PF_1,78423.33949,8.36
1,W_C,1039394,PF_2,131276.48040,8.97
2,W_A,1975221,PF_3,23069.57275,8.65
3,W_B,1396615,PF_4,53988.69284,7.89
4,W_C,1026987,PF_0,23517.17321,8.68


<a id='2'></a>
## A primer on product demand
When attempting to forecast inventory, there are two big forces that will ultimately drive how much a company will be able and desire to have on hand.  The first force is production capabilities.  For the purposes of this lecture we will assume that the manufacturing sites are built to adequatly produce any needed amount of inventory, and that raw materials are not in short supply. 

The other driving force is product demand.  If people are only buying a few hundred units from a company, it would be silly to manufacture millions.  On the other hand, if demand for the product is going off the charts, a company will need to make sure it sustains enough inventory to satisfy its customers, and not miss out on potential sales. Companies will typically dedicate huge amounts of resources to study market sentiment and guage what product demand will be.  For this lecture, a hypothetical demand forecast is provided.   

In [4]:
#Read in demand forecast
demand_data = pd.read_csv('Demand_Plan.csv')

In [5]:
#Get a feel for the structure of the data
demand_data.head()

,Unique_ID,SKU_ID,Warehouse,Product_Family,Demand,Year,Month,Weeks in Month
0,11395072,1395072,W_B,PF_1,17631,2022,1,4
1,11039394,1039394,W_C,PF_2,15438,2022,1,4
2,11975221,1975221,W_A,PF_3,12725,2022,1,4
3,11396615,1396615,W_B,PF_4,38768,2022,1,4
4,11026987,1026987,W_C,PF_0,44662,2022,1,4


When working with seperate sets of data, a great thing to start with is to see what relationships the two data sets have.  In the above two data sets, we can see that both data sets show a warehouse, SKU, and product family.  This will be important when merging the two sets for analysis.

<a id='3'></a>
##  Data Manipulation 

In [6]:
#Next, let's restructure the demand data so that we can get a weekly forecast from the monthly one.  We will
#assume a standard 4-4-5 calendar.

week_map = pd.read_csv('Week_Mapper.csv')



master = pd.DataFrame()
for i in range(1,13):
    temp = demand_data[demand_data['Month'] == i]
    temp = temp.merge(week_map, on = ['Month'], how = 'outer')
    temp = temp.dropna()
    master = master.append(temp)

    
master['Demand'] = master['Demand'] / master['Weeks in Month']
scrubbed_demand = pd.DataFrame(columns = ['SKU_ID','Warehouse','Product_Family','Year'])
for i in range(1,53):
    temp = master[master['Week']==i]
    temp = temp.drop(columns = ['Week','Month','Weeks in Month','Unique_ID'])
    temp = temp.rename(columns = {'Demand':'Week_' + str(i) + '_Demand'})
    scrubbed_demand = scrubbed_demand.merge(temp,on = ['SKU_ID','Warehouse','Product_Family','Year'],how = 'outer')
    

scrubbed_demand.head()

,SKU_ID,Warehouse,Product_Family,Week_1_Demand,Year,Week_2_Demand,Week_3_Demand,Week_4_Demand,Week_5_Demand,Week_6_Demand,...,Week_43_Demand,Week_44_Demand,Week_45_Demand,Week_46_Demand,Week_47_Demand,Week_48_Demand,Week_49_Demand,Week_50_Demand,Week_51_Demand,Week_52_Demand
0,1395072.0,W_B,PF_1,4407.75,2022.0,4407.75,4407.75,4407.75,7921.75,7921.75,...,3687.75,5930.25,5930.25,5930.25,5930.25,2899.2,2899.2,2899.2,2899.2,2899.2
1,1039394.0,W_C,PF_2,3859.50,2022.0,3859.50,3859.50,3859.50,4833.25,4833.25,...,3408.50,11672.75,11672.75,11672.75,11672.75,5741.0,5741.0,5741.0,5741.0,5741.0
2,1975221.0,W_A,PF_3,3181.25,2022.0,3181.25,3181.25,3181.25,7850.75,7850.75,...,5954.25,4810.50,4810.50,4810.50,4810.50,2729.2,2729.2,2729.2,2729.2,2729.2
3,1396615.0,W_B,PF_4,9692.00,2022.0,9692.00,9692.00,9692.00,12068.50,12068.50,...,8563.50,11172.00,11172.00,11172.00,11172.00,11519.6,11519.6,11519.6,11519.6,11519.6
4,1026987.0,W_C,PF_0,11165.50,2022.0,11165.50,11165.50,11165.50,4251.50,4251.50,...,3259.25,8376.50,8376.50,8376.50,8376.50,6810.8,6810.8,6810.8,6810.8,6810.8


<a id='4'></a>
## A note on Weeks of Supply
When measuring a company's inventory, it is often helpful to look at both absolte measures of inventory (such as number of units) as well as relative measures of inventory such as weeks of supply.  In general, weeks of supply is calculated as inventory in week 0 divided by the average of the demand for that inventory in weeks 1 ,2 ,3 , & 4.  This relative measure will allow us to compare relative efficient use of various products as well as areas that need attention.  

In [7]:
#Merge in initial inventory positions.

scrubbed_demand = scrubbed_demand.merge(inventory_data, on = ['Warehouse','SKU_ID','Product_Family'], how = 'left')
scrubbed_demand.head()



,SKU_ID,Warehouse,Product_Family,Week_1_Demand,Year,Week_2_Demand,Week_3_Demand,Week_4_Demand,Week_5_Demand,Week_6_Demand,...,Week_45_Demand,Week_46_Demand,Week_47_Demand,Week_48_Demand,Week_49_Demand,Week_50_Demand,Week_51_Demand,Week_52_Demand,Inventory as of 1/1/22,Cost
0,1395072.0,W_B,PF_1,4407.75,2022.0,4407.75,4407.75,4407.75,7921.75,7921.75,...,5930.25,5930.25,5930.25,2899.2,2899.2,2899.2,2899.2,2899.2,78423.33949,8.36
1,1039394.0,W_C,PF_2,3859.50,2022.0,3859.50,3859.50,3859.50,4833.25,4833.25,...,11672.75,11672.75,11672.75,5741.0,5741.0,5741.0,5741.0,5741.0,131276.48040,8.97
2,1975221.0,W_A,PF_3,3181.25,2022.0,3181.25,3181.25,3181.25,7850.75,7850.75,...,4810.50,4810.50,4810.50,2729.2,2729.2,2729.2,2729.2,2729.2,23069.57275,8.65
3,1396615.0,W_B,PF_4,9692.00,2022.0,9692.00,9692.00,9692.00,12068.50,12068.50,...,11172.00,11172.00,11172.00,11519.6,11519.6,11519.6,11519.6,11519.6,53988.69284,7.89
4,1026987.0,W_C,PF_0,11165.50,2022.0,11165.50,11165.50,11165.50,4251.50,4251.50,...,8376.50,8376.50,8376.50,6810.8,6810.8,6810.8,6810.8,6810.8,23517.17321,8.68


In [8]:
# Add in week 1 estimate of weeks of supply
scrubbed_demand['Week_1_WOS'] =  scrubbed_demand['Inventory as of 1/1/22'] / scrubbed_demand[
['Week_2_Demand','Week_3_Demand','Week_4_Demand','Week_5_Demand']].mean(axis = 1)
scrubbed_demand = scrubbed_demand.rename(columns = {'Inventory as of 1/1/22':'Week_0_Inventory'})
scrubbed_demand.to_csv('test.csv')

<a id='5'></a>
## A note on production planning
As mentioned earlier, we will make the assumption that there is no supply chain chrisis that will prevent us from purchases the necessary raw materials to manufacture our goods, BUT, we don't want to buy so much that we risk our inventory expiring.  Weeks of supply is a great guage to see if we have enough or too much of any one item.  We will assume that we want to have 8 weeks of supply of each item we have on the books.  

Note that in the below cell, there is a recursive calculaiton occurring.  This is because each of our components is dependent on the other in some respect.  Weeks of supply depends on current demand and inventory, inventory depends on production and demand, and production depends on weeks of supply and demand.  

In [10]:
wos_target = 8
for i in range(1,53):
    try:
        scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand']].mean(axis = 1))

        scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+'_Inventory'] - scrubbed_demand['Week_'+str(i)+'Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
        
        scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand','Week_'+str(i+5)+'_Demand']].mean(axis = 1)
    except:
        try:
            
            scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand']].mean(axis = 1))

            scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
            scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand']].mean(axis = 1)
        except:
            try:
                scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand']].mean(axis = 1))
                scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand']].mean(axis = 1)
            except:
                try:
                    scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand']].mean(axis = 1))
                    scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                    scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i)+'_Demand']].mean(axis = 1)
                except:
                    try:
                        scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i)+'_Demand']].mean(axis = 1))
                        scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                    except:
                        print('Error')
scrubbed_demand.to_csv('Forecast_Output.csv')

<a id='6'></a>
## Preliminary Results & Commentary
We now have a raw view of our forecest for 2022 inventory, production and weeks of supply.   In the next step we will move to Tableau to see some effective ways to present the information as well as some provide color to what the key takeaways are for this exercise.  Please review the "Forecast_Output.csv" to review the structure of the data and make it easier to follow along the 